# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /private/var/folders/xv/tqmny9tn5j32fvx9dbzq02540000gn/T/pip-req-build-gx8meqmn
  Running command git clone -q git://github.com/Theano/Theano.git /private/var/folders/xv/tqmny9tn5j32fvx9dbzq02540000gn/T/pip-req-build-gx8meqmn
  fatal: unable to connect to github.com:
  github.com[0: 140.82.113.4]: errno=Operation timed out

ERROR: Command errored out with exit status 128: git clone -q git://github.com/Theano/Theano.git /private/var/folders/xv/tqmny9tn5j32fvx9dbzq02540000gn/T/pip-req-build-gx8meqmn Check the logs for full command output.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 102.7MB 123kB/s eta 0:00:01    |█████████▌                      | 30.6MB 409kB/s eta 0:02:57     |███████████▎                    | 36.4MB 467kB/s eta 0:02:22     |█████████████                   | 42.0MB 409kB/s eta 0:02:29     |██████████████████████████████  | 96.3MB 411kB/s eta 0:00:16
   

  Using cached https://files.pythonhosted.org/packages/b9/63/df50cac98ea0d5b006c55a399c3bf1db9da7b5a24de7890bc9cfd5dd9e99/certifi-2019.11.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/1e/a94a8d635fa3ce4cfc7f506003548d0a2447ae76fd5ca53932970fe3053f/pyasn1-0.4.8-py2.py3-none-any.whl
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp37-none-any.whl size=61682 sha256=8100d894757dd46a54b1982ec4c65c2fadcaffd626fcb2e19091c52482152aa0
  Stored in directory: /Users/lucasfellipe/Library/Caches/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Created wheel for wrapt: filename=wrapt-1.11.2-cp37-cp37m-macosx_10_9_x86_64.whl size=32378 sha256=7481cefbcb95270600683a86232eed3656517b95e5a1aea4c6e1e1465f89e959
  Stored in directory: /Users/lucasfellipe/Library/Caches/pip/wheels/d

ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.23.0 which is incompatible.
  Found existing installation: pip 19.2.2
    Uninstalling pip-19.2.2:
      Successfully uninstalled pip-19.2.2
  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2
  Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1


# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'softsign'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


# Ajustando a ANN ao conjunto de treinamento

In [11]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 286us/step - loss: 0.5115 - acc: 0.7951
Epoch 2/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.4186 - acc: 0.8006 0s - loss: 0
Epoch 3/100
8000/8000 [==============================] - 2s 242us/step - loss: 0.4074 - acc: 0.8057
Epoch 4/100
8000/8000 [==============================] - 2s 212us/step - loss: 0.3981 - acc: 0.8100
Epoch 5/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.3900 - acc: 0.8145
Epoch 6/100
8000/8000 [==============================] - 2s 251us/step - loss: 0.3845 - acc: 0.8222
Epoch 7/100
8000/8000 [==============================] - 2s 263us/step - loss: 0.3791 - acc: 0.8327
Epoch 8/100
8000/8000 [==============================] - 2s 220us/step - loss: 0.3757 - acc: 0.8369
Epoch 9/100
8000/8000 [==============================] - 2s 216us/step - loss: 0.3738 - acc: 0.8381
Epoch 10/100
8000/8000 [===============

8000/8000 [==============================] - 1s 169us/step - loss: 0.3392 - acc: 0.8650
Epoch 78/100
8000/8000 [==============================] - 1s 157us/step - loss: 0.3356 - acc: 0.8655
Epoch 79/100
8000/8000 [==============================] - 1s 182us/step - loss: 0.3348 - acc: 0.8654
Epoch 80/100
8000/8000 [==============================] - 1s 174us/step - loss: 0.3347 - acc: 0.8629
Epoch 81/100
8000/8000 [==============================] - 2s 211us/step - loss: 0.3343 - acc: 0.8639
Epoch 82/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.3333 - acc: 0.8660
Epoch 83/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.3348 - acc: 0.8655
Epoch 84/100
8000/8000 [==============================] - 1s 159us/step - loss: 0.3347 - acc: 0.8651 0s - loss: 0.3374 -
Epoch 85/100
8000/8000 [==============================] - 1s 157us/step - loss: 0.3346 - acc: 0.8646
Epoch 86/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.33

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [8]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [9]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [14]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'softplus'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 190us/step - loss: 0.5625 - acc: 0.7172
Epoch 2/100
8000/8000 [==============================] - 1s 171us/step - loss: 0.4791 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 181us/step - loss: 0.4644 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 186us/step - loss: 0.4511 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 194us/step - loss: 0.4409 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4346 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 155us/step - loss: 0.4310 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.4296 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 2s 189us/step - loss: 0.4286 - acc: 0.7959 0s - loss: 0.4294 - acc: 0.795
Epoch 10/100
8000/800

8000/8000 [==============================] - 1s 176us/step - loss: 0.3961 - acc: 0.8371
Epoch 78/100
8000/8000 [==============================] - 2s 199us/step - loss: 0.3961 - acc: 0.8367
Epoch 79/100
8000/8000 [==============================] - 1s 140us/step - loss: 0.3961 - acc: 0.8366
Epoch 80/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.3960 - acc: 0.8374
Epoch 81/100
8000/8000 [==============================] - 1s 144us/step - loss: 0.3960 - acc: 0.8371
Epoch 82/100
8000/8000 [==============================] - 1s 159us/step - loss: 0.3959 - acc: 0.8369
Epoch 83/100
8000/8000 [==============================] - 1s 174us/step - loss: 0.3958 - acc: 0.8377
Epoch 84/100
8000/8000 [==============================] - 1s 184us/step - loss: 0.3956 - acc: 0.8372
Epoch 85/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.3956 - acc: 0.8375
Epoch 86/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.3953 - acc: 0.8369
Epo

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [20]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 246us/step - loss: 0.5237 - acc: 0.7952
Epoch 2/100
8000/8000 [==============================] - 1s 169us/step - loss: 0.4490 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 189us/step - loss: 0.4366 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 214us/step - loss: 0.4309 - acc: 0.7959
Epoch 5/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.4287 - acc: 0.7981
Epoch 6/100
8000/8000 [==============================] - 1s 175us/step - loss: 0.4277 - acc: 0.8001
Epoch 7/100
8000/8000 [==============================] - 1s 150us/step - loss: 0.4271 - acc: 0.8020
Epoch 8/100
8000/8000 [==============================] - 1s 176us/step - loss: 0.4272 - acc: 0.8021
Epoch 9/100
8000/8000 [==============================] - 1s 175us/step - loss: 0.4268 - acc: 0.8042
Epoch 10/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4264 - acc: 0.8057

8000/8000 [==============================] - 2s 194us/step - loss: 0.3978 - acc: 0.8357
Epoch 81/100
8000/8000 [==============================] - 1s 177us/step - loss: 0.3975 - acc: 0.8355
Epoch 82/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.3973 - acc: 0.8349
Epoch 83/100
8000/8000 [==============================] - 1s 186us/step - loss: 0.3972 - acc: 0.8362
Epoch 84/100
8000/8000 [==============================] - 1s 180us/step - loss: 0.3972 - acc: 0.8356 0s - loss: 0.
Epoch 85/100
8000/8000 [==============================] - 2s 201us/step - loss: 0.3970 - acc: 0.8357
Epoch 86/100
8000/8000 [==============================] - 2s 246us/step - loss: 0.3967 - acc: 0.8361
Epoch 87/100
8000/8000 [==============================] - 1s 180us/step - loss: 0.3968 - acc: 0.8366
Epoch 88/100
8000/8000 [==============================] - 2s 188us/step - loss: 0.3967 - acc: 0.8354
Epoch 89/100
8000/8000 [==============================] - 1s 177us/step - loss: 0.3964 - a

In [12]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%f\nmean:%f\nvariance:%f.format(accuracies, mean, variance))

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [15]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'softsign'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'softsign'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
7200/7200 [==============================] - 4s 508us/step - loss: 3.2706 - acc: 0.7971
Epoch 2/50
7200/7200 [==============================] - 6s 867us/step - loss: 3.2706 - acc: 0.7971
Epoch 3/50
7200/7200 [==============================] - 1s 190us/step - loss: 3.2706 - acc: 0.7971
Epoch 4/50
7200/7200 [==============================] - 1s 156us/step - loss: 3.2706 - acc: 0.7971
Epoch 5/50
7200/7200 [==============================] - 1s 154us/step - loss: 3.2706 - acc: 0.7971
Epoch 6/50
7200/7200 [==============================] - 1s 147us/step - loss: 3.2706 - acc: 0.7971
Epoch 7/50
7200/7200 [==============================] - 4s 522us/step - loss: 3.2706 - acc: 0.7971
Epoch 8/50
7200/7200 [==============================] - 4s 579us/step - loss: 3.2706 - acc: 0.7971
Epoch 9/50
7200/7200 [=======================

7200/7200 [==============================] - 6s 857us/step - loss: 0.3721 - acc: 0.8374
Epoch 26/50
7200/7200 [==============================] - 4s 618us/step - loss: 0.3757 - acc: 0.8356
Epoch 27/50
7200/7200 [==============================] - 4s 600us/step - loss: 0.3724 - acc: 0.8390
Epoch 28/50
7200/7200 [==============================] - 4s 585us/step - loss: 0.3693 - acc: 0.8401
Epoch 29/50
7200/7200 [==============================] - 4s 592us/step - loss: 0.3715 - acc: 0.8408
Epoch 30/50
7200/7200 [==============================] - 4s 589us/step - loss: 0.3731 - acc: 0.8367
Epoch 31/50
7200/7200 [==============================] - 4s 592us/step - loss: 0.3719 - acc: 0.8383
Epoch 32/50
7200/7200 [==============================] - 4s 601us/step - loss: 0.3714 - acc: 0.8404
Epoch 33/50
7200/7200 [==============================] - 4s 596us/step - loss: 0.3743 - acc: 0.8407
Epoch 34/50
7200/7200 [==============================] - 4s 604us/step - loss: 0.3703 - acc: 0.8399
Epoch 35/50


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)